In [2]:
import pandas as pd
import os
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2022.

**Q1 Read the data for January. How many columns are there?**

In [3]:
taxi_yellow_january = pd.read_parquet(os.path.join(os.getcwd(), 'data/yellow_tripdata_2022-01.parquet'))
taxi_yellow_february = pd.read_parquet(os.path.join(os.getcwd(), 'data/yellow_tripdata_2022-02.parquet'))

In [4]:
print(f'There are {len(taxi_yellow_january.columns)} columns.')

There are 19 columns.


Now let's compute the duration variable. It should contain the duration of a ride in minutes.

**Q2 What's the standard deviation of the trips duration in January?**

In [5]:
taxi_yellow_january['duration'] = taxi_yellow_january.tpep_dropoff_datetime - taxi_yellow_january.tpep_pickup_datetime
taxi_yellow_january['duration'] = taxi_yellow_january['duration'].apply(lambda td: td.total_seconds() / 60)

In [6]:
print(f'Standard deviation of the trips in January {taxi_yellow_january.duration.std()}')

Standard deviation of the trips in January 46.44530513776499


Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

**Q3 What fraction of the records left after you dropped the outliers?**

In [7]:
taxi_yellow_january_wo_outliers  = taxi_yellow_january[(taxi_yellow_january.duration>=1)&(taxi_yellow_january.duration<=60)]

In [8]:
print(f'The fraction of the records left {len(taxi_yellow_january_wo_outliers)/len(taxi_yellow_january)}')

The fraction of the records left 0.9827547930522406


Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries
- Fit a dictionary vectorizer
- Get a feature matrix from it

**Q4 What's the dimensionality of this matrix (number of columns)?**

In [9]:
cat_features = ['PULocationID', 'DOLocationID']
target = ['duration']

In [10]:
taxi_yellow_january_wo_outliers[cat_features] = taxi_yellow_january_wo_outliers[cat_features].astype(str)

train_dicts = taxi_yellow_january_wo_outliers[cat_features].to_dict(orient='records')

_dict_vectorizer = DictVectorizer()
X_train = _dict_vectorizer.fit_transform(train_dicts)

/tmp/ipykernel_158270/2806768490.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxi_yellow_january_wo_outliers[cat_features] = taxi_yellow_january_wo_outliers[cat_features].astype(str)


In [11]:
print(f'There are {X_train.shape[1]} columns.')

There are 515 columns.


Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data

**Q5 What's the RMSE on train?**

In [12]:
y_train = taxi_yellow_january_wo_outliers[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)


In [13]:
print(f'RMSE equals {mean_squared_error(y_train, y_pred, squared=False)}')

RMSE equals 6.986190135962495


Now let's apply this model to the validation dataset (February 2022).

**Q6 What's the RMSE on validation?**

In [14]:
taxi_yellow_february['duration'] = taxi_yellow_february.tpep_dropoff_datetime - taxi_yellow_february.tpep_pickup_datetime
taxi_yellow_february['duration'] = taxi_yellow_february['duration'].apply(lambda td: td.total_seconds() / 60)

In [15]:
#apply the same outlier filtering as for training
taxi_yellow_february  = taxi_yellow_february[(taxi_yellow_february.duration>=1)&(taxi_yellow_february.duration<=60)]

In [16]:
taxi_yellow_february[cat_features] = taxi_yellow_february[cat_features].astype(str)

In [17]:
test_dicts = taxi_yellow_february[cat_features].to_dict(orient='records')

X_test = _dict_vectorizer.transform(test_dicts)

In [18]:
y_test = taxi_yellow_february[target]

In [19]:
y_pred_test = lr.predict(X_test)

In [20]:
print(f'RMSE for validation equals {mean_squared_error(y_test, y_pred_test, squared=False)}')

RMSE for validation equals 7.786389417027388
